In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tables
import flammkuchen as fl
from pathlib import Path
from tqdm import tqdm
from scipy.ndimage import morphology
from lotr.data_preprocessing.suite2p_conv import export_suite2p
import tifffile as tiff
import json

In [ ]:
master =  Path(r"Z:\Hagar\e0075\v06\gad1b")

In [ ]:
paths_list = list(master.glob("*_f*/suite2p/*00*"))
unexported = [f for f in paths_list if not (f / "data_from_suite2p_unfiltered.h5").exists()]

In [ ]:
for path in tqdm(unexported):
    try:
        path = path / 'plane0'
        print(path)
        export_suite2p(path, path.parent / "data_from_suite2p_cells.h5", include_all_rois=False)
    except:
        print("Error")

In [ ]:
#Saving anatomy stacks for 2p data
files = list(master.glob("*_f*"))
for f in files:
    print(f)
    dataset_path = f / 'original'
    with open(dataset_path / "stack_metadata.json") as ff:
            stack_metadata = json.load(ff)
    n_planes, n_x, n_y = stack_metadata["shape_full"][1:]    
    
    anatomy_stack = np.zeros((n_planes, n_x, n_y))
    for plane in range(n_planes):
        try:
            plane_name = '000' + str(plane)
            path = f / 'suite2p' / plane_name

            # loading suite2p data:
            suite2p_data = fl.load(path / 'data_from_suite2p_cells.h5')
        except:
            plane_name = '00' + str(plane)
            path = f / 'suite2p' / plane_name
            suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
            
        anatomy = suite2p_data['anatomy_stack'][0]
        anatomy_stack[plane] = anatomy
    print(np.shape(anatomy_stack))
    tiff.imsave(f / "anatomy_suite2p.tif", anatomy_stack)
